In [9]:
import pandas as pd
import json
import ast

%load_ext autoreload
%autoreload 2
import sys
import pandas as pd
sys.path.append("..")
from Services import GeoService, MapFeatureAggregator, AddressToCoordinateConverter


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
with open("../building_type_mapping.json", 'r') as file:
    building_type_mapping = json.load(file)

In [3]:
myrealty = pd.read_csv("../scraping_results/data/myrealty_apartments.csv")
bars = pd.read_csv("../scraping_results/data/bars_apartments.csv")


In [4]:
bnakaran = pd.read_csv("../scraping_results/data/bnakaran_apartments.csv")
bnakaran = bnakaran.dropna(axis = 1, how = "all")
bnakaran["building_type"] = bnakaran["construction_type"].map(
    building_type_mapping["bnakaran"]
)
bnakaran["bathroom_count"] = [di.get("bathrooms", None) for di in bnakaran["room_details"].apply(ast.literal_eval).to_list()]

bnakaran.head()

,source,id,price,area,rooms,floor,storeys,building_type,added_in_date,additional_features,...,longitude,flooring,entrance_door,construction_type,renovation,windows,heating,parking,cooling,bathroom_count
0,bnakaran,d54284,1100.0,46,2,7,11,Panel,17.11.2023,"['air conditioner', 'new wiring', 'new water t...",...,40.185799,NaN,metal,wall building,"renovated, uninhabited",plastic,gas boiler,outdoor,air conditioner,1
1,bnakaran,d111383,155000.0,130,4,4,5,Stone,16.11.2023,"['new wiring', 'persistent water', 'hardwood f...",...,40.199857,hardwood,metal,stone,renovated,plastic,gas boiler,outdoor,NaN,2
2,bnakaran,d116405,99500.0,68,2,5,16,Panel,17.11.2023,"['persistent water', 'hardwood flooring', 'ele...",...,40.220956,hardwood,metal,wall building,normal,plastic,gas boiler,NaN,NaN,2
3,bnakaran,d117808,250000.0,126,4,3,15,Monolit,17.11.2023,"['new wiring', 'intercom', 'new water tubes', ...",...,40.171077,NaN,metal,in situ concrete,not renovated,plastic,NaN,NaN,NaN,2
4,bnakaran,d111336,800000.0,320,5,18,18,Monolit,16.11.2023,"['heating flooring', 'new wiring', 'new water ...",...,40.199896,laminate,metal,in situ concrete,euro renovated,plastic,centralized,underground,NaN,5


In [5]:
final_columns = ["price", "area", "rooms", "storeys", "floor", "building_type", "latitude", "longitude"]

In [6]:
bnakaran_clean = bnakaran[final_columns]
bnakaran_clean = pd.concat([
    bnakaran_clean,
    pd.get_dummies(bnakaran_clean["building_type"])
], axis = 1)
bnakaran_clean = bnakaran_clean.drop(columns = ["building_type"])
bnakaran_clean.head(3)

,price,area,rooms,storeys,floor,latitude,longitude,Monolit,Panel,Stone
0,1100.0,46,2,11,7,44.512299,40.185799,0,1,0
1,155000.0,130,4,5,4,44.525986,40.199857,0,0,1
2,99500.0,68,2,16,5,44.492191,40.220956,0,1,0


In [105]:
price_whole = bnakaran_clean[bnakaran_clean["price"] > 15000]
price_whole["price"] = price_whole["price"] / price_whole["area"]
price_whole['coordinates'] = price_whole.apply(lambda row: (row['longitude'], row['latitude']), axis=1)
price_whole = price_whole.reset_index()
price_whole = price_whole.drop(columns = ["index"])
price_whole.head()

/var/folders/67/47xpns1s0zn6h8g8npsy5_q80000gn/T/ipykernel_88307/3638294461.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_whole["price"] = price_whole["price"] / price_whole["area"]
/var/folders/67/47xpns1s0zn6h8g8npsy5_q80000gn/T/ipykernel_88307/3638294461.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_whole['coordinates'] = price_whole.apply(lambda row: (row['longitude'], row['latitude']), axis=1)


,price,area,rooms,storeys,floor,latitude,longitude,Monolit,Panel,Stone,coordinates
0,1192.307692,130,4,5,4,44.525986,40.199857,0,0,1,"(40.199857018654, 44.525986305557)"
1,1463.235294,68,2,16,5,44.492191,40.220956,0,1,0,"(40.220956, 44.492191)"
2,1984.126984,126,4,15,3,44.448088,40.171077,1,0,0,"(40.171077222759, 44.448087642852)"
3,2500.000000,320,5,18,18,44.486277,40.199896,1,0,0,"(40.199895630524, 44.486276542328)"
4,2252.747253,91,3,8,6,44.524138,40.212931,0,0,0,"(40.212930647332, 44.524138171966)"


In [10]:
significant_places = { 
    "opera" : (40.18600,44.51509),
    "malibu" : (40.18306,44.50799),
    "aua" : (40.19243,44.50446),
    "zvartnoc" : (40.1523,44.4005)
    # avan
    # cemeteries
    # try 20-30 locations
}
geo_service = GeoService(
    significant_places,
    radius = 300
)
featurer = MapFeatureAggregator(geo_service)
significant_distances = featurer.significant_distances(price_whole, "coordinates")
amenities = featurer.amenities_count(price_whole, "coordinates")

Progress: 811/811


In [106]:
price_whole

,price,area,rooms,storeys,floor,latitude,longitude,Monolit,Panel,Stone,coordinates
0,1192.307692,130,4,5,4,44.525986,40.199857,0,0,1,"(40.199857018654, 44.525986305557)"
1,1463.235294,68,2,16,5,44.492191,40.220956,0,1,0,"(40.220956, 44.492191)"
2,1984.126984,126,4,15,3,44.448088,40.171077,1,0,0,"(40.171077222759, 44.448087642852)"
3,2500.000000,320,5,18,18,44.486277,40.199896,1,0,0,"(40.199895630524, 44.486276542328)"
4,2252.747253,91,3,8,6,44.524138,40.212931,0,0,0,"(40.212930647332, 44.524138171966)"
...,...,...,...,...,...,...,...,...,...,...,...
806,4489.795918,98,3,10,9,44.513199,40.181301,1,0,0,"(40.181301116943, 44.513198852539)"
807,4358.974359,78,2,10,9,44.513199,40.181301,1,0,0,"(40.181301116943, 44.513198852539)"
808,3230.769231,65,2,4,4,44.510994,40.183401,0,0,1,"(40.183401070873, 44.510994329044)"
809,4489.795918,98,3,10,9,44.513199,40.181301,1,0,0,"(40.181301116943, 44.513198852539)"


In [107]:
significant_distances

,opera,malibu,aua,zvartnoc
0,1797,2413,2006,11900
1,4346,4422,3338,10906
2,5929,5260,5345,4551
3,2894,2627,1753,9006
4,3091,3593,2826,12480
...,...,...,...,...
806,546,483,1443,10104
807,546,483,1443,10104
808,452,258,1147,10005
809,546,483,1443,10104


In [108]:
price_whole = pd.concat([price_whole, significant_distances], axis = 1)
price_whole = pd.concat([price_whole, amenities], axis = 1)

In [109]:
price_whole = price_whole.drop(columns = ["coordinates"])

In [110]:
y = price_whole["price"]
X = price_whole.drop(columns = ["price"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf_regressor = RandomForestRegressor()
rf_regressor.fit(X_train, y_train)

RandomForestRegressor()

In [113]:
feature_importances = rf_regressor.feature_importances_

# Create a DataFrame to display feature importances alongside feature names
feature_names = X.columns  # Assuming X is your feature DataFrame
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})

# Sort the DataFrame by importance in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Print or display the feature importance DataFrame
feature_importance_df[:20]

,Feature,Importance
26,L_shelter,0.205450
0,area,0.204470
1,rooms,0.175942
10,malibu,0.088962
9,opera,0.050439
5,longitude,0.038215
31,L_parking_entrance,0.028631
25,L_restaurant,0.027618
12,zvartnoc,0.021610
2,storeys,0.020082
